<h1>Segmenting and Clustering Neighborhoods in Toronto</h1>

In [49]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

#soup.body.table.contents

for string in soup.body.table.stripped_strings:
    print(repr(string))
#soup = BeautifulSoup(source, 'lxml')
#toronto_wiki = soup.find('body')
#toronto_postals =  toronto_wiki.find('table', class_="wikitable sortable").text
#print(toronto_postals)
  

'Postcode'
'Borough'
'Neighbourhood'
'M1A'
'Not assigned'
'Not assigned'
'M2A'
'Not assigned'
'Not assigned'
'M3A'
'North York'
'Parkwoods'
'M4A'
'North York'
'Victoria Village'
'M5A'
'Downtown Toronto'
'Harbourfront'
'M5A'
'Downtown Toronto'
'Regent Park'
'M6A'
'North York'
'Lawrence Heights'
'M6A'
'North York'
'Lawrence Manor'
'M7A'
"Queen's Park"
'Not assigned'
'M8A'
'Not assigned'
'Not assigned'
'M9A'
'Etobicoke'
'Islington Avenue'
'M1B'
'Scarborough'
'Rouge'
'M1B'
'Scarborough'
'Malvern'
'M2B'
'Not assigned'
'Not assigned'
'M3B'
'North York'
'Don Mills North'
'M4B'
'East York'
'Woodbine Gardens'
'M4B'
'East York'
'Parkview Hill'
'M5B'
'Downtown Toronto'
'Ryerson'
'M5B'
'Downtown Toronto'
'Garden District'
'M6B'
'North York'
'Glencairn'
'M7B'
'Not assigned'
'Not assigned'
'M8B'
'Not assigned'
'Not assigned'
'M9B'
'Etobicoke'
'Cloverdale'
'M9B'
'Etobicoke'
'Islington'
'M9B'
'Etobicoke'
'Martin Grove'
'M9B'
'Etobicoke'
'Princess Gardens'
'M9B'
'Etobicoke'
'West Deane Park'
'M1C'
'Sca